In [39]:
import numpy as np
import pandas as pd
import os
import holoviews as hv
import hvplot.pandas
import hvplot
import geoviews as gv
from holoviews import opts
from holoviews.operation.datashader import datashade
from holoviews.streams import RangeXY, Pipe
from pyproj import CRS, Transformer


import dask as dd

import requests
import threading
import time
from threading import Event

hv.output(backend='bokeh')

# Aim

- Plot the data for unique sensors (unique sensors already available)

Following the cencus data

https://examples.pyviz.org/census/census.html

In [38]:
DATAPATH = '../data/archive.sensor.community/csv_per_month'

date = "2022-12"
sensor = "sds011"

cfile_name = sensor + '_unique.csv'
cfile_name_all = date + '_' + sensor + '.csv'

cfile_path = os.path.join(DATAPATH, date, cfile_name)
cfile_path_all = os.path.join(DATAPATH, date, cfile_name_all)

dtypes = dict(P1='object', P2='object')

df = pd.read_csv(cfile_path)
#df['timestamp'] = dd.to_datetime(df['timestamp'])

In [37]:
df.head()

,Unnamed: 0,sensor_id,lat,lon
0,0,30527,42.777134,23.147150
1,1,64777,42.485529,23.407186
2,2,21671,52.242000,8.168000
3,3,11642,50.930000,6.918000
4,4,61391,52.892000,5.738000


In [ ]:
ddf = dd.

In [4]:
class DataLoader(threading.Thread):
    """
    Fetches aggregated 5 minutes data from data.sensor.community. Sends data to holoviews pipe if supplied.
    """
    def __init__(self, wait=5, hv_pipe=None):
        super().__init__()
        self.wait = 5
        self.exit = Event()
        self.data = None
        self.url = "https://data.sensor.community/static/v2/data.dust.min.json"
        self.hv_pipe = hv_pipe
        
    def run(self):
        while not self.exit.is_set():
            data = requests.get(self.url)
            self.data = self.parse_json(data.json())
            if self.hv_pipe:
                self.hv_pipe.send(self.data)
            self.exit.wait(60*self.wait)
            
    def parse_json(self, data_json):
            data_dict = {item['sensor']['id']: {**{subitem['value_type']: subitem['value'] for subitem in item['sensordatavalues']}, 
                                            **{'sensor_type': item['sensor']['sensor_type']['name'], 'lat': item['location']['latitude'], 'lon': item['location']['longitude']}} 
                     for item in data_json}
            dt = {'P1': float, 'P2': float, 'lat': float, 'lon': float}
            return pd.DataFrame(data_dict).T.loc[:, ['P1', 'P2', 'sensor_type', 'lat', 'lon']].reset_index(names='sensor_id').astype(dtype=dt, errors='ignore')
            
    def stop(self):
        self.exit.set()
        self.join()

## Using range selector to slice dataframe and Update bins

In [5]:
background = gv.tile_sources.OSM
# transformer does reverse x and y (not using it)
transformer = Transformer.from_crs("epsg:3857", "epsg:4326")
xy_range = RangeXY(source=background)

hv_pipe = Pipe([])
data_pipe = DataLoader(hv_pipe=hv_pipe)
data_pipe.start()

In [6]:
data_pipe.data

In [35]:

# transformer does reverse x and y (not using it)
#transformer = Transformer.from_crs("epsg:3857", "epsg:4326")

background = gv.tile_sources.OSM
xy_range = RangeXY(source=background)


def hexmap(x_range, y_range):
    
    df1 = data_pipe.data
    df_sel = df1
    if x_range:
        xy_0 = hv.util.transform.easting_northing_to_lon_lat(x_range[0], y_range[0])
        xy_1 = hv.util.transform.easting_northing_to_lon_lat(x_range[1], y_range[1])

        mask1 = (df1['lon'] >=xy_0[0]) & (df1['lon']<=xy_1[0])
        mask2 = (df1['lat'] >=xy_0[1]) & (df1['lat']<=xy_1[1])

        df_sel = df1.loc[mask1 & mask2, :]

    bins = df_sel.hvplot.hexbin(x='lon',
                                y='lat',
                                C='P1',
                                alpha=0.75,
                                hover_color='pink',
                                hover_alpha=0.8,
                                gridsize=20,
                                min_count=1,
                                scale=1,
                                geo=True,
                                cmap='YlOrRd'
                               ).opts(aggregator=np.mean)

    return bins

def points(x_range, y_range, x_thresh=3):
    
    df1 = data_pipe.data
    df_sel = pd.DataFrame(dict(lon=[], lat=[]))
    if x_range:
        
        xy_0 = hv.util.transform.easting_northing_to_lon_lat(x_range[0], y_range[0])
        xy_1 = hv.util.transform.easting_northing_to_lon_lat(x_range[1], y_range[1])
        if (xy_1[0] - xy_0[0] <= x_thresh):
            # plot only points if certain zoom level is reached
            mask1 = (df1['lon'] >=xy_0[0]) & (df1['lon']<=xy_1[0])
            mask2 = (df1['lat'] >=xy_0[1]) & (df1['lat']<=xy_1[1])

            df_sel = df1.loc[mask1 & mask2, :]
        else:
            df_sel = pd.DataFrame(dict(lon=[], lat=[]))

    p = df_sel.hvplot.points(x='lon',
                            y='lat',
                            hover=[],
                            size=15,
                            geo=True)
    return p

#xy_range = RangeXY(source=background)

dyn = hv.DynamicMap(hexmap, streams=[xy_range])
dynp = hv.DynamicMap(points, streams=[xy_range])

plot_all = (background*dyn*dynp)
plot_all.opts(gv.opts(global_extent=False,
                      width=1000,
                      height=600,
                      responsive=False
                     ))

:DynamicMap   []
   :Overlay
      .WMTS.I     :WMTS   [Longitude,Latitude]
      .HexTiles.I :HexTiles   [lon,lat]   (P1)
      .Points.I   :Points   [lon,lat]